# 回归\分类概率-融合

## 简单加权平均，结果直接融合

In [2]:
## 生成一些简单的样本数据，test_prei 代表第i个模型的预测值
test_pre1 = [1.2, 3.2, 2.1, 6.2]
test_pre2 = [0.9, 3.1, 2.0, 5.9]
test_pre3 = [1.1, 2.9, 2.2, 6.0]

# y_test_true 代表第模型的真实值
y_test_true = [1, 3, 2, 6] 

In [3]:
import numpy as np
import pandas as pd

## 定义结果的加权平均函数
def Weighted_method(test_pre1,test_pre2,test_pre3,w=[1/3,1/3,1/3]):
    Weighted_result = w[0]*pd.Series(test_pre1)+w[1]*pd.Series(test_pre2)+w[2]*pd.Series(test_pre3)
    return Weighted_result

MAE（Mean Absolute Error，平均绝对误差）是一个用于衡量预测模型性能的指标，特别是在回归问题中。它计算的是预测值与真实值之间绝对误差的平均值。这里的“绝对误差”指的是预测值与真实值之差的绝对值，这样可以避免正负误差相互抵消的问题。

MAE 的值越小，说明预测模型的性能越好，因为它意味着预测值更接近真实值。

In [4]:
from sklearn import metrics
# 各模型的预测结果计算MAE
print('Pred1 MAE:',metrics.mean_absolute_error(y_test_true, test_pre1))
print('Pred2 MAE:',metrics.mean_absolute_error(y_test_true, test_pre2))
print('Pred3 MAE:',metrics.mean_absolute_error(y_test_true, test_pre3))

Pred1 MAE: 0.1750000000000001
Pred2 MAE: 0.07499999999999993
Pred3 MAE: 0.10000000000000009


In [5]:
## 根据加权计算MAE
w = [0.3,0.4,0.3] # 定义比重权值
Weighted_pre = Weighted_method(test_pre1,test_pre2,test_pre3,w)
print('Weighted_pre MAE:',metrics.mean_absolute_error(y_test_true, Weighted_pre))

Weighted_pre MAE: 0.05750000000000027


可以发现加权结果相对于之前的结果是有提升的，这种我们称其为简单的加权平均。

还有一些特殊的形式，比如mean平均，median平均

In [6]:
## 定义结果的加权平均函数
def Mean_method(test_pre1,test_pre2,test_pre3):
    Mean_result = pd.concat([pd.Series(test_pre1),pd.Series(test_pre2),pd.Series(test_pre3)],axis=1).mean(axis=1)
    return Mean_result

In [7]:
Mean_pre = Mean_method(test_pre1,test_pre2,test_pre3)
print('Mean_pre MAE:',metrics.mean_absolute_error(y_test_true, Mean_pre))

Mean_pre MAE: 0.06666666666666693


In [8]:
## 定义结果的加权平均函数
def Median_method(test_pre1,test_pre2,test_pre3):
    Median_result = pd.concat([pd.Series(test_pre1),pd.Series(test_pre2),pd.Series(test_pre3)],axis=1).median(axis=1)
    return Median_result

In [9]:
Median_pre = Median_method(test_pre1,test_pre2,test_pre3)
print('Median_pre MAE:',metrics.mean_absolute_error(y_test_true, Median_pre))

Median_pre MAE: 0.07500000000000007


## Stacking融合（回归）

In [10]:
from sklearn import linear_model

def Stacking_method(train_reg1,train_reg2,train_reg3,y_train_true,test_pre1,test_pre2,test_pre3,model_L2= linear_model.LinearRegression()):
    model_L2.fit(pd.concat([pd.Series(train_reg1),pd.Series(train_reg2),pd.Series(train_reg3)],axis=1).values,y_train_true)
    Stacking_result = model_L2.predict(pd.concat([pd.Series(test_pre1),pd.Series(test_pre2),pd.Series(test_pre3)],axis=1).values)
    return Stacking_result

In [11]:
## 生成一些简单的样本数据，test_prei 代表第i个模型的预测值
train_reg1 = [3.2, 8.2, 9.1, 5.2]
train_reg2 = [2.9, 8.1, 9.0, 4.9]
train_reg3 = [3.1, 7.9, 9.2, 5.0]
# y_test_true 代表第模型的真实值
y_train_true = [3, 8, 9, 5] 

test_pre1 = [1.2, 3.2, 2.1, 6.2]
test_pre2 = [0.9, 3.1, 2.0, 5.9]
test_pre3 = [1.1, 2.9, 2.2, 6.0]

# y_test_true 代表第模型的真实值
y_test_true = [1, 3, 2, 6] 

In [12]:
model_L2= linear_model.LinearRegression()
Stacking_pre = Stacking_method(train_reg1,train_reg2,train_reg3,y_train_true,
                               test_pre1,test_pre2,test_pre3,model_L2)
print('Stacking_pre MAE:',metrics.mean_absolute_error(y_test_true, Stacking_pre))

Stacking_pre MAE: 0.04213483146067476


可以发现模型结果相对于之前有进一步的提升，这是我们需要注意的一点是，对于第二层Stacking的模型不宜选取的过于复杂，这样会导致模型在训练集上过拟合，从而使得在测试集上并不能达到很好的效果。

# 分类模型融合

In [19]:
from sklearn.datasets import make_blobs
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons
from sklearn.metrics import accuracy_score,roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

XGBoost（eXtreme Gradient Boosting）是一种优化的分布式梯度增强库，旨在实现高效、灵活和便携的机器学习算法。以下是关于XGBoost的详细介绍：

一、概述
定义：XGBoost是一个开源的软件库，在梯度提升框架下执行优化的分布式梯度提升机器学习算法。它是对梯度提升算法的改进，求解损失函数极值时使用了牛顿法，将损失函数泰勒展开到二阶，并加入了正则化项以提高模型的泛化能力。
应用：XGBoost广泛用于回归、分类和排名等机器学习问题，是数据科学竞赛中的热门工具，如Kaggle上的许多结构化数据竞赛。

二、算法原理
加法模型：XGBoost是一种加法模型，包含多个基学习器（通常是决策树），每个基学习器对输入样本进行预测，并将预测结果相加作为最终输出。
目标函数：XGBoost的目标函数由两部分组成：损失函数和正则化项。损失函数用于衡量预测值与真实值之间的差异，而正则化项则用于控制模型的复杂度，防止过拟合。
二阶泰勒展开：在求解目标函数时，XGBoost使用了二阶泰勒展开来近似表达损失函数，这使得求解过程更加精确和高效。

三、特性与优势
高效性：XGBoost提供了并行树提升功能，能够显著加快模型的训练速度。同时，它支持在多种分布式环境（如Hadoop、SGE、MPI）上运行，解决了大规模数据处理的难题。
灵活性：XGBoost支持多种基学习器和损失函数，可以根据不同的应用场景和数据特性进行选择和优化。此外，它还提供了丰富的参数配置选项，允许用户根据需要进行精细调整。
准确性：由于XGBoost在求解过程中使用了二阶泰勒展开和正则化项，因此它能够生成更加准确和稳定的预测模型。这使得XGBoost在许多机器学习竞赛中脱颖而出，成为数据科学家和机器学习工程师的首选工具之一。

## Voting投票机制

硬投票（Hard Voting）
定义：
硬投票是一种基于多数表决的投票策略。在硬投票中，每个基本学习器（或称为基本模型）都对样本进行预测，并投票选择出现次数最多的类别作为最终的预测结果。如果存在平局，则通常选择其中一个类别作为结果（具体选择哪个类别可能取决于实现方式或额外规则）。

特点：
直观简单：硬投票通过直接统计各个学习器预测结果中的类别票数来做出决策，易于理解和实现。
依赖票数：最终的预测结果完全依赖于各个学习器预测结果的票数分布，不考虑预测概率或置信度。

软投票（Soft Voting）
定义：
软投票是一种基于概率的投票策略。在软投票中，每个基本学习器都会给出每个类别的概率或置信度估计，然后对这些概率进行平均或加权平均，以产生最终的预测结果。最终的类别是概率平均值最高的那个类别。

特点：
考虑概率：软投票不仅考虑了学习器预测的类别，还考虑了预测的概率或置信度，因此能够更全面地利用学习器的输出信息。
准确性可能更高：由于考虑了概率信息，软投票在多数情况下能够比硬投票产生更准确的预测结果。特别是在各个学习器预测结果的置信度差异较大时，软投票的优势更为明显。

In [20]:
'''
硬投票：对多个模型直接进行投票，不区分模型结果的相对重要度，最终投票数最多的类为最终被预测的类。
'''
iris = datasets.load_iris()

x=iris.data
y=iris.target
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

clf1 = XGBClassifier(learning_rate=0.1, n_estimators=150, max_depth=3, min_child_weight=2, subsample=0.7,
                     colsample_bytree=0.6, objective='binary:logistic')
clf2 = RandomForestClassifier(n_estimators=50, max_depth=1, min_samples_split=4,
                              min_samples_leaf=63,oob_score=True)
clf3 = SVC(C=0.1)

# 硬投票
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('svc', clf3)], voting='hard')
for clf, label in zip([clf1, clf2, clf3, eclf], ['XGBBoosting', 'Random Forest', 'SVM', 'Ensemble']):
    scores = cross_val_score(clf, x, y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.95 (+/- 0.02) [XGBBoosting]
Accuracy: 0.33 (+/- 0.00) [Random Forest]
Accuracy: 0.92 (+/- 0.03) [SVM]
Accuracy: 0.95 (+/- 0.02) [Ensemble]


XGBoost（XGBBoosting）:
准确率非常高，达到了 0.95，标准差也很小（0.02），说明模型在不同交叉验证折上的表现非常稳定。

随机森林（Random Forest）:
准确率异常低，只有 0.33，这明显不符合预期。这可能是由于随机森林的参数设置不当导致的过拟合或欠拟合问题。特别地，max_depth=1 和 min_samples_leaf=63 的组合可能非常严格，导致树无法有效地分割数据。通常，max_depth 应该设置得更高，而 min_samples_leaf 应该设置得更低，以便模型能够学习到数据的复杂模式。

支持向量机（SVM）:
准确率也较高，为 0.92，但略低于 XGBoost。SVM 在高维数据和小数据集上通常表现良好，但可能不如一些集成方法（如 XGBoost）灵活。

集成模型（Ensemble）:
使用硬投票的集成模型（VotingClassifier）达到了与 XGBoost 相似的准确率（0.95），这显示了集成方法的强大之处。尽管随机森林的性能非常差，但其他两个模型的良好表现足以弥补这一点，因为硬投票机制会简单地选择多数投票的类别。

In [21]:
'''
软投票：和硬投票原理相同，增加了设置权重的功能，可以为不同模型设置不同权重，进而区别模型不同的重要度。
'''
x=iris.data
y=iris.target
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3)

clf1 = XGBClassifier(learning_rate=0.1, n_estimators=150, max_depth=3, min_child_weight=2, subsample=0.8,
                     colsample_bytree=0.8, objective='binary:logistic')
clf2 = RandomForestClassifier(n_estimators=50, max_depth=1, min_samples_split=4,
                              min_samples_leaf=63,oob_score=True)
clf3 = SVC(C=0.1, probability=True)

# 软投票
eclf = VotingClassifier(estimators=[('xgb', clf1), ('rf', clf2), ('svc', clf3)], voting='soft', weights=[2, 1, 1])
clf1.fit(x_train, y_train)

for clf, label in zip([clf1, clf2, clf3, eclf], ['XGBBoosting', 'Random Forest', 'SVM', 'Ensemble']):
    scores = cross_val_score(clf, x, y, cv=5, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (scores.mean(), scores.std(), label))

Accuracy: 0.95 (+/- 0.02) [XGBBoosting]
Accuracy: 0.33 (+/- 0.00) [Random Forest]
Accuracy: 0.92 (+/- 0.03) [SVM]
Accuracy: 0.95 (+/- 0.02) [Ensemble]


## 分类的Stacking\Blending融合

Stacking，有时也被称为stacked generalization或堆叠泛化，是机器学习中的一种集成学习方法。它通过组合多个模型的预测结果来提高整体的预测性能。Stacking的基本流程包括以下几个步骤：

一、基本概念

定义：Stacking是指训练一个模型用于组合其他各个模型的预测结果，即首先训练多个不同的基学习器（base learners），然后将这些基学习器的输出作为输入来训练一个元学习器（meta-learner），以得到一个最终的输出。

结构：Stacking模型通常由两层组成，第一层是多个基学习器，第二层是用于组合基学习器输出的元学习器。

二、工作流程

数据划分：将原始数据集划分为训练集和测试集（或验证集），通常使用交叉验证等方法来更充分地利用训练数据。

训练基学习器：在训练集上独立训练多个不同的基学习器，这些基学习器可以是不同类型的模型，如决策树、神经网络、支持向量机等。

生成一级预测结果：使用训练好的基学习器对训练集（或另一个独立的数据集）进行预测，得到每个基学习器的预测结果。

构造次级学习器：将基学习器的预测结果作为新的特征输入到次级学习器中，次级学习器的目标是结合这些预测结果来生成最终的预测。

训练次级学习器：使用基学习器的预测结果和原始数据的标签来训练次级学习器。

最终预测：使用训练好的次级学习器对测试集进行预测，得到最终的预测结果。

这段代码实现了一个基于5折交叉验证的Stacking集成学习框架，用于提高分类任务的性能。Stacking是一种集成学习技术，它将多个模型的预测结果作为新特征，然后使用另一个模型（称为元模型或次级模型）基于这些新特征进行最终预测。以下是代码的详细解释：

数据准备：
假设代码中有iris数据集（尽管没有直接导入，这里假设是sklearn.datasets中的Iris数据集）。
从Iris数据集中取出前100个样本作为训练和测试的数据集（Iris数据集总共有150个样本，这里可能是为了简化问题或模拟一个更小的数据集）。

定义基模型：
初始化了一个包含5个不同分类器的列表clfs，包括逻辑回归（但注意代码中实际未导入LogisticRegression，这里假设它已经被导入）、随机森林、两个不同标准的额外树分类器、梯度提升分类器。

数据切分：
使用train_test_split函数将数据切分为训练集（X, y）和测试集（X_predict, y_predict），测试集大小为30%（test_size=0.3）。

初始化用于Stacking的数据集：
dataset_blend_train和dataset_blend_test分别用于存储训练集和测试集上各基模型的预测结果，作为新的特征。

5折Stacking：
使用StratifiedKFold进行5折交叉验证，确保每一折中各类别的比例与整体数据集一致。
对于每个基模型，在每次交叉验证的每一折中，用训练集的一部分训练模型，并用训练好的模型对剩余的训练集部分进行预测，得到该部分的预测结果作为新特征。
对于测试集，使用每个基模型在所有训练数据上训练后得到的模型进行预测，并将这些预测结果取平均，作为测试集的新特征。
注意：这里有一个小问题，即在计算测试集上的新特征时，dataset_blend_test_j[:, i]的更新方式实际上是不正确的，因为它在每个内部循环中都被重新计算，而应该是累积每个折叠的预测结果后再取平均。正确的做法应该是在内部循环外部计算平均值。

训练元模型：
使用逻辑回归作为元模型，以dataset_blend_train（包含所有基模型的预测结果作为新特征）作为输入，y作为目标变量进行训练。

预测和评估：
使用训练好的元模型对dataset_blend_test（包含测试集上所有基模型的预测结果作为新特征）进行预测，得到最终的预测结果。
使用ROC AUC分数评估元模型在测试集上的性能。

In [22]:
'''
5-Fold Stacking
'''
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier,GradientBoostingClassifier
import pandas as pd
#创建训练的数据集
data_0 = iris.data
data = data_0[:100,:]

target_0 = iris.target
target = target_0[:100]

#模型融合中使用到的各个单模型
clfs = [LogisticRegression(solver='lbfgs'),
        RandomForestClassifier(n_estimators=5, n_jobs=-1, criterion='gini'),
        ExtraTreesClassifier(n_estimators=5, n_jobs=-1, criterion='gini'),
        ExtraTreesClassifier(n_estimators=5, n_jobs=-1, criterion='entropy'),
        GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=5)]
 
#切分一部分数据作为测试集
X, X_predict, y, y_predict = train_test_split(data, target, test_size=0.3, random_state=2020)

dataset_blend_train = np.zeros((X.shape[0], len(clfs)))
dataset_blend_test = np.zeros((X_predict.shape[0], len(clfs)))

#5折stacking
n_splits = 5
skf = StratifiedKFold(n_splits)
skf = skf.split(X, y)

for j, clf in enumerate(clfs):
    #依次训练各个单模型
    dataset_blend_test_j = np.zeros((X_predict.shape[0], 5))
    for i, (train, test) in enumerate(skf):
        #5-Fold交叉训练，使用第i个部分作为预测，剩余的部分来训练模型，获得其预测的输出作为第i部分的新特征。
        X_train, y_train, X_test, y_test = X[train], y[train], X[test], y[test]
        clf.fit(X_train, y_train)
        y_submission = clf.predict_proba(X_test)[:, 1]
        dataset_blend_train[test, j] = y_submission
        dataset_blend_test_j[:, i] = clf.predict_proba(X_predict)[:, 1]
    #对于测试集，直接用这k个模型的预测值均值作为新的特征。
    dataset_blend_test[:, j] = dataset_blend_test_j.mean(1)
    print("val auc Score: %f" % roc_auc_score(y_predict, dataset_blend_test[:, j]))

clf = LogisticRegression(solver='lbfgs')
clf.fit(dataset_blend_train, y)
y_submission = clf.predict_proba(dataset_blend_test)[:, 1]

print("Val auc Score of Stacking: %f" % (roc_auc_score(y_predict, y_submission)))

val auc Score: 1.000000
val auc Score: 0.500000
val auc Score: 0.500000
val auc Score: 0.500000
val auc Score: 0.500000
Val auc Score of Stacking: 1.000000


在您的输出结果中，val auc Score: 1.000000 和 val auc Score: 0.500000 的极端值可能表明了一些潜在的问题或误解。首先，让我们澄清一些概念和可能的错误源：

AUC Score 的范围：AUC（Area Under the Curve）分数是ROC曲线下的面积，其值范围是从0到1。一个完美的模型将具有AUC分数为1.0，而一个随机猜测的模型（即没有预测能力的模型）将具有AUC分数接近0.5。

极端值的原因：
AUC = 1.000000：这通常意味着模型完美地预测了所有样本的标签，这在某些情况下是可能的，但在大多数实际应用中很少见，尤其是在处理复杂数据集时。如果这是基于测试集（或验证集）的结果，并且数据不是完全分离的，那么这可能是过拟合的迹象，或者测试集和训练集之间的某种形式的数据泄漏。
AUC = 0.500000：这通常表示模型没有提供比随机猜测更好的预测能力。这可能是因为模型没有正确训练，或者模型类型与数据不匹配。

在机器学习中，blending（混合）是一种集成学习技术，它类似于stacking但有一些关键的区别。Blending通常涉及将多个模型的预测结果以某种方式组合起来，以产生最终的预测。这种组合可以是简单的平均（如算术平均或加权平均），也可以是更复杂的聚合方法，如投票、堆叠（尽管堆叠通常指一个更复杂的过程）或学习一个元模型来预测最佳输出。

Blending 与 Stacking 的主要区别：

数据集划分：
Stacking：在stacking中，整个数据集被划分为训练集和测试集（或验证集）。然后，训练集被进一步划分为多个子训练集和子测试集（或子验证集），用于交叉验证。每个子训练集用于训练一个基模型，而对应的子测试集用于生成该基模型对测试集的预测（这些预测随后被用作元模型的输入特征）。
Blending：在blending中，数据集通常被明确划分为两部分：一部分用于训练所有基模型（称为“blending训练集”），另一部分用于生成基模型的预测（这些预测随后被用作元模型的输入特征），但不用于训练基模型本身（这部分数据通常被称为“blending测试集”或“holdout集”）。

元模型的训练：
在两种方法中，元模型都是使用基模型的预测结果作为输入特征进行训练的。但在blending中，元模型是在一个与训练基模型完全不同的数据集上训练的，这有助于减少过拟合的风险。

复杂度：
从实现的角度来看，blending通常比stacking更简单，因为它不需要进行复杂的交叉验证步骤来生成基模型的预测。然而，这也可能意味着blending在利用数据方面没有stacking那么高效，因为它没有充分利用所有数据来训练基模型。

Blending 的实施步骤：

数据划分：将数据集划分为两部分：blending训练集和blending测试集（或holdout集）。

训练基模型：使用blending训练集训练多个基模型。

生成预测：使用训练好的基模型对blending测试集进行预测，生成预测结果。

训练元模型：将基模型的预测结果作为输入特征，使用blending测试集的真实标签来训练元模型。

最终预测：使用训练好的元模型对新的数据实例进行预测，或者如果需要，可以将基模型和元模型的预测结果以某种方式组合起来（尽管在典型的blending设置中，只使用元模型的预测）。

需要注意的是，blending和stacking并不是互斥的，而且在实际应用中，它们之间的界限可能并不总是那么清晰。此外，还有其他集成学习方法，如bagging和boosting，它们也使用多个模型来改进预测性能，但具有不同的实现方式和目标。

这段代码实现了机器学习中的blending集成学习方法，特别地，它使用了多个基模型（也称为弱学习器）的预测结果作为新特征来训练一个元模型（也称为强学习器）。以下是对这段代码的详细解释：

数据准备：
使用iris数据集（尽管代码中未直接导入iris，但假设iris.data和iris.target已经以某种方式被加载了，通常是通过from sklearn.datasets import load_iris; iris = load_iris()）。
选择数据集的前100个样本作为训练和验证的数据集（因为iris数据集有150个样本，但这里只使用了前两种类别的样本，共100个）。

定义基模型：
定义了五个基模型，包括逻辑回归（LogisticRegression）、随机森林（RandomForestClassifier，使用基尼不纯度和熵作为分裂标准各一个）、极端随机树（ExtraTreesClassifier，使用基尼不纯度）和梯度提升树（GradientBoostingClassifier）。

数据切分：
将数据进一步切分为训练集（X, y）和测试集（X_predict, y_predict），其中测试集占总数据的30%。
接着，将训练集再次切分为d1（用于训练基模型）和d2（用于生成基模型的预测结果，这些预测结果将作为元模型的输入特征）。

训练基模型并生成预测结果：
遍历所有基模型，对每个模型使用d1数据集进行训练。
然后，对每个模型使用d2数据集（训练集的另一半）和测试集X_predict生成预测概率（对于二分类问题，通常关注第二个类别的概率）。
这些预测概率被存储在dataset_d1和dataset_d2中，分别对应于d2和测试集的预测结果。
注意：这里打印了每个基模型在测试集上的AUC分数，但这并不是blending过程的实际验证，因为blending的验证应该在元模型级别进行。

训练元模型：
使用dataset_d1（包含d2的预测结果）和对应的标签y_d2来训练元模型（在这里是另一个梯度提升树模型）。
元模型训练完成后，使用它对dataset_d2（包含测试集的预测结果）进行预测，生成最终的预测概率。

评估blending模型的性能：
最后，计算并打印了blending模型在测试集上的AUC分数。这是通过比较元模型在测试集上的预测概率y_submission和真实的测试集标签y_predict来完成的。

In [23]:
'''
Blending
'''
 
#创建训练的数据集
#创建训练的数据集
data_0 = iris.data
data = data_0[:100,:]

target_0 = iris.target
target = target_0[:100]
 
#模型融合中使用到的各个单模型
clfs = [LogisticRegression(solver='lbfgs'),
        RandomForestClassifier(n_estimators=5, n_jobs=-1, criterion='gini'),
        RandomForestClassifier(n_estimators=5, n_jobs=-1, criterion='entropy'),
        ExtraTreesClassifier(n_estimators=5, n_jobs=-1, criterion='gini'),
        #ExtraTreesClassifier(n_estimators=5, n_jobs=-1, criterion='entropy'),
        GradientBoostingClassifier(learning_rate=0.05, subsample=0.5, max_depth=6, n_estimators=5)]
 
#切分一部分数据作为测试集
X, X_predict, y, y_predict = train_test_split(data, target, test_size=0.3, random_state=2020)

#切分训练数据集为d1,d2两部分
X_d1, X_d2, y_d1, y_d2 = train_test_split(X, y, test_size=0.5, random_state=2020)
dataset_d1 = np.zeros((X_d2.shape[0], len(clfs)))
dataset_d2 = np.zeros((X_predict.shape[0], len(clfs)))
 
for j, clf in enumerate(clfs):
    #依次训练各个单模型
    clf.fit(X_d1, y_d1)
    y_submission = clf.predict_proba(X_d2)[:, 1]
    dataset_d1[:, j] = y_submission
    #对于测试集，直接用这k个模型的预测值作为新的特征。
    dataset_d2[:, j] = clf.predict_proba(X_predict)[:, 1]
    print("val auc Score: %f" % roc_auc_score(y_predict, dataset_d2[:, j]))

#融合使用的模型
clf = GradientBoostingClassifier(learning_rate=0.02, subsample=0.5, max_depth=6, n_estimators=30)
clf.fit(dataset_d1, y_d2)
y_submission = clf.predict_proba(dataset_d2)[:, 1]
print("Val auc Score of Blending: %f" % (roc_auc_score(y_predict, y_submission)))

val auc Score: 1.000000
val auc Score: 1.000000
val auc Score: 1.000000
val auc Score: 1.000000
val auc Score: 1.000000
Val auc Score of Blending: 1.000000


# 示例

In [24]:
import pandas as pd
import numpy as np
import warnings
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
%matplotlib inline

import itertools
import matplotlib.gridspec as gridspec
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB 
from sklearn.ensemble import RandomForestClassifier
# from mlxtend.classifier import StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split
# from mlxtend.plotting import plot_learning_curves
# from mlxtend.plotting import plot_decision_regions

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split

from sklearn import linear_model
from sklearn import preprocessing
from sklearn.svm import SVR
from sklearn.decomposition import PCA,FastICA,FactorAnalysis,SparsePCA

import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import GridSearchCV,cross_val_score
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error

In [30]:
path = 'C:/Users/hp/Desktop/data/'
Train_data = pd.read_csv(path+'used_car_train_20200313.csv', sep=' ')
TestA_data = pd.read_csv(path+'used_car_testA_20200313.csv', sep=' ')
print('Train data shape:',Train_data.shape)
print('TestA data shape:',Test_data.shape)

Train data shape: (150000, 31)
TestA data shape: (50000, 30)


In [26]:
Train_data.head()

,SaleID,name,regDate,model,brand,bodyType,fuelType,gearbox,power,kilometer,...,v_5,v_6,v_7,v_8,v_9,v_10,v_11,v_12,v_13,v_14
0,0,736,20040402,30.0,6,1.0,0.0,0.0,60,12.5,...,0.235676,0.101988,0.129549,0.022816,0.097462,-2.881803,2.804097,-2.420821,0.795292,0.914762
1,1,2262,20030301,40.0,1,2.0,0.0,0.0,0,15.0,...,0.264777,0.121004,0.135731,0.026597,0.020582,-4.900482,2.096338,-1.030483,-1.722674,0.245522
2,2,14874,20040403,115.0,15,1.0,0.0,0.0,163,12.5,...,0.251410,0.114912,0.165147,0.062173,0.027075,-4.846749,1.803559,1.565330,-0.832687,-0.229963
3,3,71865,19960908,109.0,10,0.0,0.0,1.0,193,15.0,...,0.274293,0.110300,0.121964,0.033395,0.000000,-4.509599,1.285940,-0.501868,-2.438353,-0.478699
4,4,111080,20120103,110.0,5,1.0,0.0,0.0,68,5.0,...,0.228036,0.073205,0.091880,0.078819,0.121534,-1.896240,0.910783,0.931110,2.834518,1.923482


In [27]:
numerical_cols = Train_data.select_dtypes(exclude = 'object').columns
print(numerical_cols)

Index(['SaleID', 'name', 'regDate', 'model', 'brand', 'bodyType', 'fuelType',
       'gearbox', 'power', 'kilometer', 'regionCode', 'seller', 'offerType',
       'creatDate', 'price', 'v_0', 'v_1', 'v_2', 'v_3', 'v_4', 'v_5', 'v_6',
       'v_7', 'v_8', 'v_9', 'v_10', 'v_11', 'v_12', 'v_13', 'v_14'],
      dtype='object')


In [28]:
feature_cols = [col for col in numerical_cols if col not in ['SaleID','name','regDate','price']]

In [31]:
X_data = Train_data[feature_cols]
Y_data = Train_data['price']

X_test  = TestA_data[feature_cols]

print('X train shape:',X_data.shape)
print('X test shape:',X_test.shape)

X train shape: (150000, 26)
X test shape: (50000, 26)


In [32]:
def Sta_inf(data):
    print('_min',np.min(data))
    print('_max:',np.max(data))
    print('_mean',np.mean(data))
    print('_ptp',np.ptp(data))
    print('_std',np.std(data))
    print('_var',np.var(data))

In [33]:
print('Sta of label:')
Sta_inf(Y_data)

Sta of label:
_min 11
_max: 99999
_mean 5923.327333333334
_ptp 99988
_std 7501.973469876438
_var 56279605.94272992


In [34]:
X_data = X_data.fillna(-1)
X_test = X_test.fillna(-1)

In [35]:
def build_model_lr(x_train,y_train):
    reg_model = linear_model.LinearRegression()
    reg_model.fit(x_train,y_train)
    return reg_model

def build_model_ridge(x_train,y_train):
    reg_model = linear_model.Ridge(alpha=0.8)#alphas=range(1,100,5)
    reg_model.fit(x_train,y_train)
    return reg_model

def build_model_lasso(x_train,y_train):
    reg_model = linear_model.LassoCV()
    reg_model.fit(x_train,y_train)
    return reg_model

def build_model_gbdt(x_train,y_train):
    estimator =GradientBoostingRegressor(loss='ls',subsample= 0.85,max_depth= 5,n_estimators = 100)
    param_grid = { 
            'learning_rate': [0.05,0.08,0.1,0.2],
            }
    gbdt = GridSearchCV(estimator, param_grid,cv=3)
    gbdt.fit(x_train,y_train)
    print(gbdt.best_params_)
    # print(gbdt.best_estimator_ )
    return gbdt

def build_model_xgb(x_train,y_train):
    model = xgb.XGBRegressor(n_estimators=120, learning_rate=0.08, gamma=0, subsample=0.8,\
        colsample_bytree=0.9, max_depth=5) #, objective ='reg:squarederror'
    model.fit(x_train, y_train)
    return model

def build_model_lgb(x_train,y_train):
    estimator = lgb.LGBMRegressor(num_leaves=63,n_estimators = 100)
    param_grid = {
        'learning_rate': [0.01, 0.05, 0.1],
    }
    gbm = GridSearchCV(estimator, param_grid)
    gbm.fit(x_train, y_train)
    return gbm

In [36]:
## xgb
xgr = xgb.XGBRegressor(n_estimators=120, learning_rate=0.1, subsample=0.8,\
        colsample_bytree=0.9, max_depth=7) # ,objective ='reg:squarederror'

scores_train = []
scores = []

## 5折交叉验证方式
sk=StratifiedKFold(n_splits=5,shuffle=True,random_state=0)
for train_ind,val_ind in sk.split(X_data,Y_data):
    
    train_x=X_data.iloc[train_ind].values
    train_y=Y_data.iloc[train_ind]
    val_x=X_data.iloc[val_ind].values
    val_y=Y_data.iloc[val_ind]
    
    xgr.fit(train_x,train_y)
    pred_train_xgb=xgr.predict(train_x)
    pred_xgb=xgr.predict(val_x)
    
    score_train = mean_absolute_error(train_y,pred_train_xgb)
    scores_train.append(score_train)
    score = mean_absolute_error(val_y,pred_xgb)
    scores.append(score)

print('Train mae:',np.mean(score_train))
print('Val mae',np.mean(scores))

Train mae: 587.2198646903912
Val mae 685.938454869345


In [37]:
## Split data with val
x_train,x_val,y_train,y_val = train_test_split(X_data,Y_data,test_size=0.3)

## Train and Predict
print('Predict LR...')
model_lr = build_model_lr(x_train,y_train)
val_lr = model_lr.predict(x_val)
subA_lr = model_lr.predict(X_test)

print('Predict Ridge...')
model_ridge = build_model_ridge(x_train,y_train)
val_ridge = model_ridge.predict(x_val)
subA_ridge = model_ridge.predict(X_test)

print('Predict Lasso...')
model_lasso = build_model_lasso(x_train,y_train)
val_lasso = model_lasso.predict(x_val)
subA_lasso = model_lasso.predict(X_test)

print('Predict GBDT...')
model_gbdt = build_model_gbdt(x_train,y_train)
val_gbdt = model_gbdt.predict(x_val)
subA_gbdt = model_gbdt.predict(X_test)

Predict LR...
Predict Ridge...
Predict Lasso...
Predict GBDT...


KeyboardInterrupt: 

In [38]:
print('predict XGB...')
model_xgb = build_model_xgb(x_train,y_train)
val_xgb = model_xgb.predict(x_val)
subA_xgb = model_xgb.predict(X_test)

print('predict lgb...')
model_lgb = build_model_lgb(x_train,y_train)
val_lgb = model_lgb.predict(x_val)
subA_lgb = model_lgb.predict(X_test)

predict XGB...
predict lgb...
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004264 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4716
[LightGBM] [Info] Number of data points in the train set: 84000, number of used features: 24
[LightGBM] [Info] Start training from score 5904.740881
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004370 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4718
[LightGBM] [Info] Number of data points in the train set: 84000, number of used features: 24
[LightGBM] [Info] Start training from score 5900.114774
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.004372 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4714
[LightGBM] [Info] Number of data points in the train set: 84000, number of used feat

In [39]:
print('Sta inf of lgb:')
Sta_inf(subA_lgb)

Sta inf of lgb:
_min -171.3459372780405
_max: 88945.80489886308
_mean 5925.756010922327
_ptp 89117.15083614111
_std 7368.986002823645
_var 54301954.7098108


## 加权融合

In [40]:
def Weighted_method(test_pre1,test_pre2,test_pre3,w=[1/3,1/3,1/3]):
    Weighted_result = w[0]*pd.Series(test_pre1)+w[1]*pd.Series(test_pre2)+w[2]*pd.Series(test_pre3)
    return Weighted_result

## Init the Weight
w = [0.3,0.4,0.3]

## 测试验证集准确度
val_pre = Weighted_method(val_lgb,val_xgb,val_gbdt,w)
MAE_Weighted = mean_absolute_error(y_val,val_pre)
print('MAE of Weighted of val:',MAE_Weighted)

## 预测数据部分
subA = Weighted_method(subA_lgb,subA_xgb,subA_gbdt,w)
print('Sta inf:')
Sta_inf(subA)
## 生成提交文件
sub = pd.DataFrame()
sub['SaleID'] = X_test.index
sub['price'] = subA
sub.to_csv('./sub_Weighted.csv',index=False)

NameError: name 'val_gbdt' is not defined

In [41]:
## 与简单的LR（线性回归）进行对比
val_lr_pred = model_lr.predict(x_val)
MAE_lr = mean_absolute_error(y_val,val_lr_pred)
print('MAE of lr:',MAE_lr)

MAE of lr: 2603.003297279443
